Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def ImgShow(images, resolution=120):
    plt.rcParams['figure.dpi'] = resolution
    
    cnt = len(images)
    if cnt > 20:
        # Top-level object is an image, make it an array of 1
        images = [images]
        cnt = len(images)
        
    cols = cnt
    fig, ax = plt.subplots(1, cnt)    
    if 1 == cnt:
        ax = [ax]
    for i in range(0, len(images)):
        dims = len(images[i].shape)
        if 3 == dims:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        else:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
            
        ax[i].axis('off')
        ax[i].imshow(imgDisp)

In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/IMG_4430.jpg")

In [ ]:
#Find the edges using flood, start with a blur
imageBlur = cv.GaussianBlur(imageOriginal, (25,25), cv.BORDER_DEFAULT)

imageFlood = imageBlur.copy()
cols = imageFlood.shape[1]
rows = imageFlood.shape[0]
seed = (int(cols / 2), int(rows / 2)) # a point in the middle

imageFloodMask = np.zeros((rows+2,cols+2,1), np.uint8)
tolerance = 32.0
diff = (tolerance,tolerance,tolerance)
#flags = 8 | cv.FLOODFILL_MASK_ONLY | 1 << 8
flags = 8 | cv.FLOODFILL_FIXED_RANGE | 255 << 8
res = cv.floodFill(imageFlood, imageFloodMask, seed, (0,0,255),diff,diff, flags)

ImgShow([imageOriginal, imageBlur, imageFlood, imageFloodMask],200)

Find the line segments

In [ ]:
imageCanny = cv.Canny(imageFloodMask, 100, 175)

#Create default parametrization LSD
lsd = cv.createLineSegmentDetector(0)


#Detect lines in the image
lines = lsd.detect(imageCanny)[0] #Position 0 of the returned tuple are the detected lines
    
imageWithLines = lsd.drawSegments(imageCanny, lines)
ImgShow([imageWithLines])

In [ ]:
img = imageCanny
lsd = cv.createLineSegmentDetector()

if len(img.shape) == 3:
    img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

lines, width, prec, nfa = lsd.detect(img)
edgeMap = lsd.drawSegments(img, lines)[..., -1]
edgeMap2 = lsd.drawSegments(img, lines)
lines = np.squeeze(lines, 1)
edgeList = np.concatenate([lines, width, prec, nfa], 1)
ImgShow([edgeMap,edgeMap2])